In [2]:
import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
from modules.datasets.ImageContrastiveLossShopeeDataset import ImageContrastiveLossShopeeDataset
from modules.losses.CurriculumLoss import CurriculumLoss
from modules.models.SiameseNet import SiameseNet
from modules.models.ResNet18EmbeddingsShopeeNet import ResNet18EmbeddingsShopeeNet
import modules.utils.dataset_utils as dataset_utils

import os

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
DATA_FOLDER = './shopee-product-matching/'

train_df = dataset_utils.get_dataset(DATA_FOLDER, is_test=False)
train_df = dataset_utils.add_target(train_df)
train_df.head()

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,./shopee-product-matching/train_images/0000a68...,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]"
1,train_3386243561,./shopee-product-matching/train_images/0003978...,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]"
2,train_2288590299,./shopee-product-matching/train_images/000a190...,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]"
3,train_2406599165,./shopee-product-matching/train_images/00117e4...,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]"
4,train_3369186413,./shopee-product-matching/train_images/00136d1...,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]"


In [4]:
train_cl_df = dataset_utils.get_contrastive_loss_dataset(train_df)
train_cl_df = train_cl_df.sample(frac=1)

11014it [00:02, 3872.07it/s]


In [9]:

images_dataset = ImageContrastiveLossShopeeDataset(
    train_cl_df['image_1'].values,
    train_cl_df['image_2'].values,
    train_cl_df['label'].values,
    transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

train_percent = 0.8

total_len = len(images_dataset)
train_len = int(train_percent * total_len)
val_len = total_len - train_len
train_dataset, val_dataset = torch.utils.data.random_split(images_dataset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=24,
    shuffle=True,
    num_workers=1
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=24,
    shuffle=True,
    num_workers=1
)

In [10]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0
    i = 1
    tk0 = tqdm(train_loader, total=len(train_loader))
    for input_1, input_2, target in tk0:
        input_1, input_2, target = input_1.to(device), input_2.to(device), target.to(device)
        optimizer.zero_grad()
        output_1, output_2 = model(input_1, input_2)
        loss = criterion(output_1, output_2, target)
        loss.backward()
        optimizer.step()
        running_loss += loss
        running_loss /= i
        i += 1
        tk0.set_postfix(Train_Loss=running_loss,Epoch=epoch,LR=optimizer.param_groups[0]['lr'])

        # if batch_idx % 100 == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(input_1), len(train_loader.dataset),
        #                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():
        for input_1, input_2, target in tk0:
            input_1, input_2, target = input_1.to(device), input_2.to(device), target.to(device)
            output_1, output_2 = model(input_1, input_2)
            test_loss += criterion(output_1, output_2, target, reduction='sum').item()  # sum up batch loss
            # pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [14]:
device = torch.device("cpu")

resnet18_emb_model = ResNet18EmbeddingsShopeeNet().to(device)
siamese_net = SiameseNet(resnet18_emb_model).to(device)

criterion = CurriculumLoss()
optimizer = optim.Adam(siamese_net.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
num_epochs = 5

In [15]:
for epoch in range(1, num_epochs):
    train(siamese_net, device, train_loader, optimizer, criterion,epoch)
    test(siamese_net, device, val_loader, criterion)
    scheduler.step()

  0%|          | 0/5584 [00:10<?, ?it/s]


TypeError: CurriculumLoss.forward() missing 1 required positional argument: 'epoch'